In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install pretrainedmodels

In [ ]:
#!git clone https://github.com/Cadene/pretrained-models.pytorch.git
#!cd pretrained-models.pytorch
#!python setup.py install

In [ ]:
!pip install utils

In [ ]:
!pip install pytorchcv

In [ ]:
from torchvision.models import *
from pytorchcv.model_provider import get_model as ptcv_get_model
import pretrainedmodels
import torch
from fastai.vision import *
from fastai.vision.models import *
from fastai.vision.learner import model_meta

from utils import *
import sys


In [ ]:
[k for k,v in sys.modules['torchvision.models'].__dict__.items() if callable(v)]

In [ ]:
pretrainedmodels.model_names

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.vision import *
from fastai.metrics import error_rate
from fastai import *

In [ ]:
df = pd.read_csv('/kaggle/input/dataset/train.csv', header = "infer")
df

In [ ]:
df_test = pd.read_csv('/kaggle/input/dataset/test_vc2kHdQ.csv', header = "infer")
df_test

In [ ]:
from pathlib import Path

In [ ]:
path = Path('/kaggle/input/dataset/train_SOaYf6m/images/')

In [ ]:
np.random.seed(123)

In [ ]:
size = 112
tfms = get_transforms(do_flip = True,flip_vert = False,max_zoom = 1.1,max_rotate = 45, max_warp = 0.1)
data = ImageDataBunch.from_df(path, df, size=size, bs = 8, ds_tfms = tfms, seed =1, num_workers=1)
test_data = ImageList.from_df(df_test, path)
data.add_test(test_data)
#data.normalize(mean = torch.Tensor([0.485, 0.456, 0.406]), std = torch.Tensor([0.229, 0.224, 0.225]))
data.normalize(imagenet_stats)
data

In [ ]:
data.show_batch(rows=3, figsize=(7,6))

In [ ]:
def inceptionv4(pretrained=False):
    pretrained = 'imagenet' if pretrained else None
    model = pretrainedmodels.inceptionv4(pretrained=pretrained)
    all_layers = list(model.children())
    return nn.Sequential(*all_layers[0], *all_layers[1:])

In [ ]:
def xception(pretrained=False):
    pretrained = 'imagenet' if pretrained else None
    model = pretrainedmodels.xception(pretrained=pretrained)
    return nn.Sequential(*list(model.children()))

In [ ]:
def inceptionresnetv2(pretrained=False):
    pretrained = 'imagenet' if pretrained else None
    model = pretrainedmodels.inceptionresnetv2(pretrained=pretrained)
    return nn.Sequential(*model.children())

In [ ]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
def resnext101_32x4d(pretrained=False):
    pretrained = 'imagenet' if pretrained else None
    model = pretrainedmodels.resnext101_32x4d(pretrained=pretrained)
    all_layers = list(model.children())
    return nn.Sequential(*all_layers[0], *all_layers[1:])

In [ ]:
def efficientnet_b3(pretrained=False):
    return ptcv_get_model("efficientnet_b3", pretrained=False).features

In [ ]:
seed = 42

# python RNG
import random
random.seed(seed)

# pytorch RNGs
import torch
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
import numpy as np
np.random.seed(seed)

Trying EfficientNet Model

In [ ]:
learn = cnn_learner(data, models.densenet169, metrics=accuracy)
#gcontext = ssl.SSLContext()
#learn = cnn_learner(data, resnext101_32x4d, pretrained=True, cut=-2, split_on=lambda m: (m[0][6], m[1]), metrics = accuracy)
#learn = cnn_learner(data, efficientnet_b3, pretrained=True, cut=noop, split_on=lambda m: (m[0][4], m[1]), metrics = accuracy)
#learn = cnn_learner(data, inceptionv4, pretrained=True, cut=-2, split_on=lambda m: (m[0][11], m[1]), metrics = accuracy)
#learn = cnn_learner(data, inceptionresnetv2, pretrained=True, cut=-2, split_on=lambda m: (m[0][9], m[1]), metrics = accuracy)

In [ ]:
learn.model_dir = Path('/kaggle/working')

In [ ]:
learn.lr_find()
learn.recorder.plot(suggestion = True)

In [ ]:
learn.fit_one_cycle(10, max_lr = slice(1e-4,1e-3))

In [ ]:
learn.save('stage-1')

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()
learn.recorder.plot(suggestion = True)

In [ ]:
learn.fit_one_cycle(8,max_lr = slice(1e-6,1e-5/2))

In [ ]:
preds,y = learn.TTA()
acc = accuracy(preds, y)
print('The validation accuracy is {} %.'.format(acc * 100))

In [ ]:
#preds, y = learn.TTA(ds_type = DatasetType.Test)

In [ ]:
#preds

In [ ]:
newTfms = get_transforms(do_flip = True,flip_vert = False,max_zoom = 1.1,  max_rotate = 40, max_warp = 0.1)
newSz = 224
newdata = ImageDataBunch.from_df(path, df, size=newSz, bs = 8, ds_tfms = newTfms, seed = 1)
test_data = ImageList.from_df(df_test, path)
newdata.add_test(test_data)
newdata.normalize(imagenet_stats)
newdata

In [ ]:
learn.data = newdata

In [ ]:
learn.freeze()

In [ ]:
learn.lr_find()
learn.recorder.plot(suggestion = True)

In [ ]:
learn.fit_one_cycle(8, max_lr = slice(1e-6, 1e-5))

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(25,max_lr = slice(1e-6,1e-5))

In [ ]:
learn.save('stage-2')

In [ ]:
preds,y = learn.TTA()
acc = accuracy(preds, y)
print('The validation accuracy is {} %.'.format(acc * 100))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(6)

In [ ]:
interp.plot_confusion_matrix()

Trying VGG19

In [ ]:
#learn2 = cnn_learner(data, inceptionresnetv2, pretrained=True, cut=-2, split_on=lambda m: (m[0][9], m[1]), metrics = accuracy)
#learn2 = cnn_learner(data, resnext101_32x4d, pretrained=True, cut=-2, split_on=lambda m: (m[0][6], m[1]), metrics = accuracy)
learn2 = cnn_learner(newdata, models.vgg19_bn, metrics = accuracy)

In [ ]:
learn2.model_dir = Path('/kaggle/working')

In [ ]:
learn2.lr_find()
learn2.recorder.plot()

In [ ]:
learn2.fit_one_cycle(15, max_lr = slice(1e-4,1e-3))

In [ ]:
#learn2.freeze_to(-2)

In [ ]:
#learn2.lr_find()
#learn2.recorder.plot()

In [ ]:
#learn2.fit_one_cycle(15, max_lr = slice(1e-5,1e-4))

In [ ]:
learn2.unfreeze()

In [ ]:
learn2.lr_find()
learn2.recorder.plot(suggestion = True)

In [ ]:
learn2.fit_one_cycle(10, max_lr = slice(1e-6,1e-5))

In [ ]:
learn2.lr_find()
learn2.recorder.plot(suggestion = True)

In [ ]:
learn2.fit_one_cycle(10, max_lr = slice(1e-6,2e-6))

In [ ]:
learn2.lr_find()
learn2.recorder.plot(suggestion = True)

In [ ]:
learn2.fit_one_cycle(15, max_lr = 1e-6)

In [ ]:
preds,y = learn2.TTA()
acc = accuracy(preds, y)
print('The validation accuracy is {} %.'.format(acc * 100))

Updating Data

In [ ]:
learn2.data = newdata

In [ ]:
learn2.freeze()
learn2.lr_find()
learn2.recorder.plot()

In [ ]:
learn2.fit_one_cycle(15, max_lr = slice(1e-5/2, 1e-5))

In [ ]:
learn2.unfreeze()
learn2.lr_find()
learn2.recorder.plot()

In [ ]:
learn2.fit_one_cycle(35, max_lr = slice(1e-6, 1e-5))

In [ ]:
preds,y = learn2.TTA()
acc = accuracy(preds, y)
print('The validation accuracy is {} %.'.format(acc * 100))

Combining Results of Inception and Xception

In [ ]:
learn.save('/kaggle/working/ResNext')
learn2.save('/kaggle/working/Resnet50')

In [ ]:
preds1, _ = learn.get_preds(ds_type = DatasetType.Test)
preds2, _ = learn2.get_preds(ds_type = DatasetType.Test)

In [ ]:
preds1

In [ ]:
preds2

In [ ]:
pred_ensemble = (preds1 + preds2)/2
pred_ensemble

In [ ]:
prob = np.array(pred_ensemble)
preds_fin = np.argmax(prob, axis = 1)
preds_fin

In [ ]:
df_test

In [ ]:
submission = pd.DataFrame({'image_names' : df_test['image_names'], 'emergency_or_not' : preds_fin})
submission

In [ ]:
submission.to_csv("/kaggle/working/submission_ensemble2.csv", index = False)

In [ ]:
submission = pd.DataFrame({'image_names' : df_test['image_names'], 'emergency_or_not' : np.argmax(preds1, axis = 1)})
submission.to_csv("/kaggle/working/submission_resnext.csv", index = False)
submission

In [ ]:
submission = pd.DataFrame({'image_names' : df_test['image_names'], 'emergency_or_not' : np.argmax(preds2, axis = 1)})
submission.to_csv("/kaggle/working/submission_densenet121.csv", index = False)
submission

In [ ]:
preds[:,0]

In [ ]:
# Predictions from ResNext
submission_resnext = pd.DataFrame({'image_names' : df_test['image_names'], 'Prob-0' : preds1[:,0], 'Prob-1' : preds1[:,1]})
submission_resnext.to_csv("/kaggle/working/submission_prob_resnext.csv", index = False)
submission_resnext

In [ ]:
# Predictions from ResNext
submission_densenet121 = pd.DataFrame({'image_names' : df_test['image_names'], 'Prob-0' : preds2[:,0], 'Prob-1' : preds2[:,1]})
submission_densenet121.to_csv("/kaggle/working/submission_prob_densenet121.csv", index = False)
submission_densenet121

In [ ]:
Error

In [ ]:
def generateSubmission(learner):
    submissions = pd.read_csv('/kaggle/input/test_vc2kHdQ.csv')
    id_list = list(submissions.image_names)
    preds,y = learner.TTA(ds_type=DatasetType.Test)
    pred_list = list(preds[:,1])
    pred_dict = dict((key, value.item()) for (key, value) in zip(learner.data.test_ds.items,pred_list))
    pred_ordered = [pred_dict[Path('/kaggle/input/train_SOaYf6m/images/' + id)] for id in id_list]
    submissions = pd.DataFrame({'image_names':id_list,'emergency_or_not':pred_ordered})
    submissions.to_csv("submission_transferLearning_{}.csv".format(pred_score),index = False)

In [ ]:
Path()

In [ ]:
submissions = pd.read_csv('/kaggle/input/test_vc2kHdQ.csv')
id_list = list(submissions.image_names)
preds,y = learn.TTA(ds_type=DatasetType.Test)
pred_list = list(preds[:,1])
pred_dict = dict((key, value.item()) for (key, value) in zip(learn.data.test_ds.items,pred_list))
pred_ordered = [pred_dict[Path('/kaggle/input/train_SOaYf6m/images/' + id)] for id in id_list]
submissions = pd.DataFrame({'image_names':id_list,'emergency_or_not':pred_ordered})
submissions.to_csv("submission_transferLearning_{}.csv".format(pred_score),index = False)

In [ ]:
learn.data.test_ds.items

In [ ]:
Error

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(4)

In [ ]:
learn.save('stage-2')

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(5)

In [ ]:
learn.save('stage-3')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10)

In [ ]:
learn.save('stage-4')